In [1]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain_visualizer import visualize
from langchain.callbacks import get_openai_callback
import langchain
import os

langchain.debug = True

db_path = os.path.abspath("./orders.db")


db = SQLDatabase.from_uri(
    f"sqlite:////{db_path}",
    include_tables=["return_policy", "category", "product", "order"],
    sample_rows_in_table_info=3,
)
print(db.table_info)


CREATE TABLE category (
	id INTEGER, 
	name VARCHAR, 
	PRIMARY KEY (id), 
	UNIQUE (name)
)

/*
3 rows from category table:
id	name
1	Electronics
2	Fashion
3	Home & Kitchen
*/


CREATE TABLE product (
	id INTEGER, 
	name VARCHAR, 
	categoryid INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(categoryid) REFERENCES category (id)
)

/*
3 rows from product table:
id	name	categoryid
1	Samsung Galaxy S21	1
2	Apple MacBook Pro	1
3	Sony 65" 4K Smart TV	1
*/


CREATE TABLE return_policy (
	id INTEGER, 
	categoryid INTEGER, 
	policy VARCHAR, 
	"return_window(days)" INTEGER, 
	name VARCHAR, 
	PRIMARY KEY (id), 
	FOREIGN KEY(categoryid) REFERENCES category (id)
)

/*
3 rows from return_policy table:
id	categoryid	policy	return_window(days)	name
1	1	Items can be returned within 30 days of purchase with the original receipt.	30	Electronics Return Policy
2	2	Clothing and accessories can be returned within 14 days of purchase in unused condition with tags at	14	Fashion Return Policy
3	3	Most items can be re

In [ ]:
llm = OpenAI(temperature=0)
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    max_iterations=30,
    return_intermediate_steps=True,
)


def agent_executor(query):
    with get_openai_callback() as cb:
        response = agent.run(query)
        print(f"Total Tokens: {cb.total_tokens}")
        print(f"Prompt Tokens: {cb.prompt_tokens}")
        print(f"Completion Tokens: {cb.completion_tokens}")
        print(f"Total Cost (USD): ${cb.total_cost}")
        return response

In [3]:
query = "List all the relation names"
agent_executor(query)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "List all the relation names"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "List all the relation names",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct sqlite query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 10 results.\nYou can order the results by a relevant column to return the most interesting examples in the database.\nNever query for all the columns from a specific table, only ask for the relevant columns given the question.\nYou h

'return_policy, order, product, category'

In [4]:
agent_executor("describe the category table")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "describe the category table"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "describe the category table",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct sqlite query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 10 results.\nYou can order the results by a relevant column to return the most interesting examples in the database.\nNever query for all the columns from a specific table, only ask for the relevant columns given the question.\nYou h

'The categories in the database are Electronics, Fashion, Home & Kitchen, Sports & Outdoors, Beauty & Personal Care, Toys & Games, Books, Grocery, Health & Wellness, Automotive, and Perishable.'

In [9]:
agent_executor("How many orders are in order table?")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "How many orders are in order table?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "How many orders are in order table?",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct sqlite query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 10 results.\nYou can order the results by a relevant column to return the most interesting examples in the database.\nNever query for all the columns from a specific table, only ask for the relevant columns

'50'

In [5]:
agent_executor("Describe order whose id is 16. What is the product and its category?")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Describe order whose id is 16. What is the product and its category?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Describe order whose id is 16. What is the product and its category?",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct sqlite query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 10 results.\nYou can order the results by a relevant column to return the most interesting examples in the database.\nNever query for all the columns f

'The product for order 16 is Olay Regenerist Serum and its category is Beauty & Personal Care.'

In [3]:
agent_executor(
    "My order id is 16. What is the product's name and return policy related to this order?"
)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "My order id is 16. What is the product's name and return policy related to this order?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "My order id is 16. What is the product's name and return policy related to this order?",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct sqlite query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 10 results.\nYou can order the results by a relevant column to return the most interesting examples in the database

'The product name related to order id 16 is Olay Regenerist Serum and the return policy is Most beauty and personal care items can be returned within 27 days of purchase.'

In [4]:
agent_executor("My order id is 12. Can I return the product today?")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "My order id is 12. Can I return the product today?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "My order id is 12. Can I return the product today?",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct sqlite query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 10 results.\nYou can order the results by a relevant column to return the most interesting examples in the database.\nNever query for all the columns from a specific table, only ask for t

'You have 29 days from the purchase date of 2023-05-12 10:55:00 to return the product.'